---

#### Importar módulos

In [6]:
import pandas as pd
import seaborn as sns
sns.set()

#### Recarregar histórico de conflitos finalizado

In [7]:
history = pd.read_excel('Resultados/histórico-conflitos-atualizado.xlsx')

---
## Conflitos de terra em cidades de fronteira

Contagems e proporções territoriais para o período de 2000 à 2021

### Pré-processamento

#### Formatar campo de identidade do município

In [3]:
history['ID'] = history['MUN'] + ' - ' + history['UF']

#### Calcular total de conflitos por cidade

In [8]:
# Listar colunas com contagems por ano
year_columns = list(filter(lambda col: '20' in col, history.columns))

# Somar colunas com contagems por ano
history['TOTAL'] = history[year_columns].sum(axis=1)

#### Calcular total de conflitos por cidade por década

In [14]:
year_columns

['CONF2000',
 'CONF2001',
 'CONF2002',
 'CONF2003',
 'CONF2004',
 'CONF2005',
 'CONF2006',
 'CONF2007',
 'CONF2008',
 'CONF2009',
 'CONF2010',
 'CONF2011',
 'CONF2012',
 'CONF2013',
 'CONF2014',
 'CONF2015',
 'CONF2016',
 'CONF2017',
 'CONF2018',
 'CONF2019',
 'CONF2020',
 'CONF2021']

In [8]:
# Listar colunas com contagems por ano
year_columns = list(filter(lambda col: '20' in col, history.columns))

# Somar colunas com contagems por ano
history['TOTAL'] = history[year_columns].sum(axis=1)

#### Criar coluna para o arco regional ao qual o estado do município pertence

In [9]:
# Mapeamento de arcos por estado
arcos = {
    'RS': "SUL",
    'SC': "SUL",
    'PR': "SUL",
    'MS': "CENTRO",
    'MT': "CENTRO",
    'PR': "CENTRO",
    'AC': 'NORTE',
    'RO': 'NORTE',
    'RR': 'NORTE',
    'PA': 'NORTE',
    'AP': 'NORTE',
    'AM': 'NORTE',
}

# Criar coluna de `arco` ao qual o município pertence
history['ARCO'] = history['UF'].map(arcos)

#### Display processed data

In [10]:
print(history.columns)
display(history.head())

Index(['id', 'CUGN', 'CUGT', 'MUN', 'UF', 'FF', 'FFSITGEO', 'FRONTEIRA',
       'MAPA DE CONFLITOS', 'RECORRENCIA-10', 'RECORRENCIA-5', 'RECORRENCIA-1',
       'CONF2000', 'CONF2001', 'CONF2002', 'CONF2003', 'CONF2004', 'CONF2005',
       'CONF2006', 'CONF2007', 'CONF2008', 'CONF2009', 'CONF2010', 'CONF2011',
       'CONF2012', 'CONF2013', 'CONF2014', 'CONF2015', 'CONF2016', 'CONF2017',
       'CONF2018', 'CONF2019', 'CONF2020', 'CONF2021', 'TOTAL', 'ARCO'],
      dtype='object')


,id,CUGN,CUGT,MUN,UF,FF,FFSITGEO,FRONTEIRA,MAPA DE CONFLITOS,RECORRENCIA-10,...,CONF2014,CONF2015,CONF2016,CONF2017,CONF2018,CONF2019,CONF2020,CONF2021,TOTAL,ARCO
0,alta floresta d'oeste - ro,1100015,1100015,Alta Floresta D'Oeste,RO,1,2,1,3,PERMANENTE,...,1,2,4,3,1,1,8,1,40,NORTE
1,ariquemes - ro,1100023,1100023,Ariquemes,RO,0,0,0,1,ESTABILIZADO,...,3,1,0,0,1,0,1,1,26,NORTE
2,cabixi - ro,1100031,1100031,Cabixi,RO,1,2,1,0,POUCOS CONFLITOS,...,0,0,2,1,0,0,0,0,4,NORTE
3,cerejeiras - ro,1100056,1100056,Cerejeiras,RO,1,3,0,0,POUCOS CONFLITOS,...,1,0,1,0,0,1,0,0,4,NORTE
4,corumbiara - ro,1100072,1100072,Corumbiara,RO,1,3,0,0,POUCOS CONFLITOS,...,2,0,0,0,0,0,1,0,10,NORTE


---
## Gerar arquivo shape com dados + geometria de cada município

#### Carregar arquivo shape com geometria de municípios do Brasil como dataframe pandas

In [11]:
# !pip install geopandas

import geopandas as gpd

# Replace 'input_shapefile.shp' with the path to your shapefile
shapefile_path = 'Dados/BR_Municipios_2021.zip'

# Read the shapefile using geopandas
geo_df = gpd.read_file(shapefile_path)

# Convert the geo-dataframe to regular dataframe
cities = pd.DataFrame(geo_df)

#### Criar campo `id` representando cidades únicas da tabela de geometria de municípios

In [12]:
cities['id'] = cities['NM_MUN'].str.strip().str.lower() + ' - ' + cities['SIGLA'].str.strip().str.lower()

#### Cruzar colunas de geometria e área

In [13]:
history_shape = cities[['id', 'AREA_KM2', 'geometry']].join(history.set_index('id'), on='id', how='inner')

---
## Salvar arquivo shape com dados e geometrias

#### Salvar arquivos shape para compressão a partir da tabela finalizada

In [29]:
import geopandas as gpd
from shapely.wkt import loads

# # Replace 'geometry' with the actual name of your geometry column
# history_shape['geometry'] = history_shape['geometry'].apply(loads)

# Now, convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(history_shape, geometry='geometry')

# Replace 'output_shapefile.shp' with the desired output shapefile path
shapefile_output_path = f'Resultados/histórico_conflitos_atualizado_shape.shp'

# Save the GeoDataFrame to a shapefile
gdf.to_file(shapefile_output_path, driver='ESRI Shapefile')

C:\Users\luisr\AppData\Local\Temp\ipykernel_28840\1100762217.py:14: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(shapefile_output_path, driver='ESRI Shapefile')


#### Comprimir arquivos em arquivo shape zip final

In [35]:
import os
import zipfile

output_folder = 'Resultados'
base_shape_name = 'histórico_conflitos_atualizado_shape'
base_shape_path = f'{output_folder}/{base_shape_name}'
output_zip_path = f'{base_shape_path}.zip'

# List of filenames to include in the ZIP archive
selected_files = [filename for filename in os.listdir(output_folder) if base_shape_name in filename]

with zipfile.ZipFile(output_zip_path, 'w') as zipf:
    for file in selected_files:
        file_path = os.path.join(output_folder, file)
        arcname = os.path.basename(file)  # Use only the filename in the archive
        zipf.write(file_path, arcname=arcname)


#### Deletar arquivos provisórios

In [37]:
import os

# List of files to delete
files_to_delete = [f'{output_folder}/{filename}' for filename in selected_files]

# Loop through the files and delete each one
for file_path in files_to_delete:
    try:
        os.remove(file_path)  # Delete the file
        print(f"Deleted: {file_path}")
    except OSError as e:
        print(f"Error deleting {file_path}: {e}")


Deleted: Resultados/histórico_conflitos_atualizado_shape.cpg
Deleted: Resultados/histórico_conflitos_atualizado_shape.dbf
Deleted: Resultados/histórico_conflitos_atualizado_shape.prj
Deleted: Resultados/histórico_conflitos_atualizado_shape.shp
Deleted: Resultados/histórico_conflitos_atualizado_shape.shx
